# Multinomial Classification using Netsaur

In [8]:
import {
  AdamOptimizer,
  Cost,
  CPU,
  DenseLayer,
  OneCycle,
  ReluLayer,
  Sequential,
  setupBackend,
  SoftmaxLayer,
  tensor,
  tensor1D,
  tensor2D,
} from "jsr:@denosaurs/netsaur@0.3.1";

import { parse } from "jsr:@std/csv";

import {
  ClassificationReport,
  useSplit,
  Matrix,
  CategoricalEncoder,
} from "jsr:@lala/appraisal@0.4.0";

In [2]:
// Read the training dataset
const _data = Deno.readTextFileSync("../datasets/iris.csv");
const data = parse(_data);
data.shift()

// Get the predictors (x) and targets (y)
const x = data.map((fl) => fl.slice(0, 4).map(Number));
const y = data.map((fl) => fl[4]);

const encoder = new CategoricalEncoder()
const encodedY = encoder.fit(y).transform(y, "f32")

[
  "sepal length",
  "sepal width",
  "petal length",
  "petal width",
  "class"
]

In [3]:
// Split the dataset for training and testing
const [[x_train, y_train], [x_test, y_test]] = useSplit(
  { ratio: [7, 3], shuffle: true },
  x,
  encodedY
);


We classified the same dataset in our SLP example too, but we weren't able to get 100% accuracy most of the time due to the `versicolor` and `virginica` classes not being linearly separable. 

We are adding a hidden layer with `ReLU` activation before our `Softmax` output layer.

With a multi-layer neural network, we can use the `ReLU` activation function to induce non-linearity in our model. This helps us accurately classify the three species.

In [5]:
// Setup the CPU backend for Netsaur
await setupBackend(CPU);

// Create a sequential neural network
const net = new Sequential({
  // Set number of minibatches to 6
  // Set size of input layer to 4
  size: [6, 4],

  // Disable logging during training
  silent: true,

  // Define each layer of the network
  layers: [
    // A dense layer with 16 neurons
    DenseLayer({ size: [16] }),
    // A ReLu activation layer
    ReluLayer(),
    // A dense layer with 3 neurons
    DenseLayer({ size: [3] }),
    // A Softmax activation layer
    SoftmaxLayer(),
  ],
  optimizer: AdamOptimizer(),
  // We are using CrossEntropy for finding cost
  cost: Cost.CrossEntropy,
  scheduler: OneCycle({ max_rate: 0.05, step_size: 50 }),
});


CPU Backend Initialized


In [18]:
const time = performance.now();
net.train(
  [
    {
      inputs: tensor2D(x_train),
      outputs: tensor(y_train.data, y_train.shape),
    },
  ],
  300,
  5,
  0.02
);
console.log(`training time: ${performance.now() - time}ms`);


training time: 28.4493000000366ms


Now that the model has been trained, we can test its performance.

In [19]:
// Calculate metrics
const res = await net.predict(tensor2D(x_test));

const y1 = encoder.untransform(
    CategoricalEncoder.fromSoftmax(
        new Matrix(res.data, {shape: res.shape})
    )
);
const y0 = encoder.untransform(y_test);
const cReport = new ClassificationReport(y0, y1);
cReport;

Class,Precision,F1Score,Recall,Support
Iris-setosa,1,1,1,17
Iris-versicolor,1,1,1,11
Iris-virginica,1,1,1,17
Accuracy,,,1,45
